In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K

In [2]:
Sagittal_T2 = np.load('/kaggle/input/image-extraction/SagittalT2.npz')

x_train = Sagittal_T2['x'][:800]
y_train = Sagittal_T2['y'][:800]

x_test = Sagittal_T2['x'][800:1000]
y_test = Sagittal_T2['y'][800:1000]

In [3]:
def mbconv_block(inputs, filters, kernel_size, strides, expansion_factor):
    # Expansion phase
    x = Conv2D(filters * expansion_factor, kernel_size=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    
    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    
    # Projection phase
    x = Conv2D(filters, kernel_size=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    # Skip connection if possible
    if strides == 1 and inputs.shape[-1] == filters:
        x = Add()([x, inputs])
    
    return x

# カスタムEfficientNetモデル
def efficientnet_custom(inputs):
    
    # 最初のConv層
    x = Conv2D(32, kernel_size=3, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    
    # MBConvブロックをいくつか追加
    x = mbconv_block(x, filters=16, kernel_size=3, strides=1, expansion_factor=1)
    x = mbconv_block(x, filters=24, kernel_size=3, strides=2, expansion_factor=6)
    x = mbconv_block(x, filters=40, kernel_size=5, strides=2, expansion_factor=6)
    
    x = GlobalAveragePooling2D()(x)
    
    return x

In [4]:
def Sagittal_T2_CNN(input_shape, r, c,):
    
    inputs = Input(shape=input_shape)
    x = efficientnet_custom(inputs)
    
    x = Dense(128, activation='relu')(x)
    x = layers.Dense(r*c, activation=None)(x)
    x = layers.Reshape((r,c))(x)
    
    output = layers.Softmax(axis=-1)(x)
    
    model = models.Model(inputs=inputs, outputs=output)
    
    return model

In [5]:
input_shape = (224, 224, 1)  # 画像のサイズ
r, c = 5, 3  # クラス数

scs_model = Sagittal_T2_CNN(input_shape, r, c)
scs_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
history = scs_model.fit(x_train,y_train,
                        epochs=30,batch_size=32,
                        validation_data=(x_test,y_test))

Epoch 1/30


I0000 00:00:1728395688.169606      64 service.cc:145] XLA service 0x7e8660011100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728395688.169677      64 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


 3/25 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.3101 - loss: 1.1006

I0000 00:00:1728395696.654798      64 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 146ms/step - accuracy: 0.6638 - loss: 0.9458 - val_accuracy: 0.8890 - val_loss: 0.9496
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.8829 - loss: 0.5244 - val_accuracy: 0.8890 - val_loss: 0.7586
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.8814 - loss: 0.4328 - val_accuracy: 0.8890 - val_loss: 0.6151
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.8764 - loss: 0.4154 - val_accuracy: 0.8890 - val_loss: 0.5224
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.8918 - loss: 0.3623 - val_accuracy: 0.8890 - val_loss: 0.4813
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.8786 - loss: 0.3794 - val_accuracy: 0.8890 - val_loss: 0.4505
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.8876 - loss: 0.3540 - val_accuracy: 0.8890 - val_loss: 0.4291
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.8790 - loss: 0.3637 - val_accuracy: 0.8890 - val_loss: 

In [7]:
pred = scs_model.predict(x_train[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [8]:
np.sum(pred[0],axis=1)

array([1.        , 0.99999994, 1.        , 1.        , 0.99999994],
      dtype=float32)

In [9]:
def Axial_T2_CNN(input_shape, r, c):
    
    inputs = Input(shape=input_shape)
    x = efficientnet_custom(inputs)
    
    xL = Dense(128, activation='relu')(x)
    xL = layers.Dense(r*c, activation=None)(x)
    xL = layers.Reshape((r,c))(xL)
    
    outputL = layers.Softmax(axis=-1, name='Left' )(xL)
    
    
    xR = Dense(128, activation='relu')(x)
    xR = layers.Dense(r*c, activation=None)(x)
    xR = layers.Reshape((r,c))(xR)
    
    outputR = layers.Softmax(axis=-1, name='Right')(xR)
    
    model = models.Model(inputs=inputs, outputs=[outputL,outputR])
    
    return model

In [10]:
Axial_T2 = np.load('/kaggle/input/image-extraction/axial_t2.npz')

axial_train_x = Axial_T2['x'][:800]
axialL_train_y = Axial_T2['y_left'][:800]
axialR_train_y = Axial_T2['y_right'][:800]

axial_test_x = Axial_T2['x'][800:1000]
axialL_test_y = Axial_T2['y_left'][800:1000]
axialR_test_y = Axial_T2['y_right'][800:1000]

In [11]:
input_shape = (224, 224, 10)  # 画像のサイズ
r, c = 5, 3  # クラス数

ss_model = Axial_T2_CNN(input_shape, r, c)
ss_model.compile(optimizer='adam',
                 loss={'Left':'categorical_crossentropy',
                       'Right':'categorical_crossentropy'},
                 metrics={'Left':'accuracy',
                          'Right':'accuracy'})

In [12]:
ss_model.fit(axial_train_x,{'Left':axialL_train_y,'Right':axialR_train_y},
             epochs=30,batch_size=32,
             validation_data=(axial_test_x,
                              {'Left':axialL_test_y,
                               'Right':axialR_test_y}
                             ))

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 211ms/step - Left_accuracy: 0.4049 - Right_accuracy: 0.3964 - loss: 2.1892 - val_Left_accuracy: 0.5520 - val_Right_accuracy: 0.6260 - val_loss: 2.1056
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - Left_accuracy: 0.6128 - Right_accuracy: 0.6094 - loss: 2.0393 - val_Left_accuracy: 0.6480 - val_Right_accuracy: 0.6260 - val_loss: 1.9788
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - Left_accuracy: 0.6745 - Right_accuracy: 0.6742 - loss: 1.8643 - val_Left_accuracy: 0.7560 - val_Right_accuracy: 0.7540 - val_loss: 1.7990
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - Left_accuracy: 0.7324 - Right_accuracy: 0.7241 - loss: 1.6499 - val_Left_accuracy: 0.7560 - val_Right_accuracy: 0.7540 - val_loss: 1.5986
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - Left_accuracy: 0.7484 - Right_accuracy: 0.7441 - loss: 1.4427 - val_Left_accuracy: 0.7560 - val_Right_accuracy: 0.7540 - val_loss: 1.4293
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88

In [13]:
def Saggital_T1_CNN(input_shape, r, c):
    
    inputsL = Input(shape=input_shape)
    inputsR = Input(shape=input_shape)
    xL = efficientnet_custom(inputsL)
    xR = efficientnet_custom(inputsR)
    
    xL = Dense(128, activation='relu')(xL)
    xL = layers.Dense(r*c, activation=None)(xL)
    xL = layers.Reshape((r,c))(xL)
    
    outputL = layers.Softmax(axis=-1, name='Left' )(xL)
    
    
    xR = Dense(128, activation='relu')(xR)
    xR = layers.Dense(r*c, activation=None)(xR)
    xR = layers.Reshape((r,c))(xR)
    
    outputR = layers.Softmax(axis=-1, name='Right')(xR)
    
    model = models.Model(inputs=[inputsL, inputsR], outputs=[outputL, outputR])
    
    return model

In [14]:
input_shape = (224, 224, 2)  # 画像のサイズ
r, c = 5, 3  # クラス数

nfn_model = Saggital_T1_CNN(input_shape, r, c)
nfn_model.compile(optimizer='adam',
                 loss={'Left':'categorical_crossentropy',
                       'Right':'categorical_crossentropy'},
                 metrics={'Left':'accuracy',
                          'Right':'accuracy'})

In [15]:
Saggital_T1L = np.load('/kaggle/input/image-extraction/left_sagittal_t1.npz')
Saggital_T1R = np.load('/kaggle/input/image-extraction/right_sagittal_t1.npz')

saggitalT1L_train_x = Saggital_T1L['x'][:800]
saggitalT1L_train_y = Saggital_T1L['y'][:800]

saggitalT1L_test_x = Saggital_T1L['x'][800:1000]
saggitalT1L_test_y = Saggital_T1L['y'][800:1000]

saggitalT1R_train_x = Saggital_T1R['x'][:800]
saggitalT1R_train_y = Saggital_T1R['y'][:800]

saggitalT1R_test_x = Saggital_T1R['x'][800:1000]
saggitalT1R_test_y = Saggital_T1R['y'][800:1000]

In [16]:
nfn_model.fit([saggitalT1L_train_x,saggitalT1R_train_x],{'Left':saggitalT1L_train_y,'Right':saggitalT1R_train_y},
             epochs=30,batch_size=32,
             validation_data=([saggitalT1L_test_x,saggitalT1R_test_x],{'Left':saggitalT1L_test_y,'Right':saggitalT1R_test_y}))

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 241ms/step - Left_accuracy: 0.5304 - Right_accuracy: 0.5613 - loss: 2.0287 - val_Left_accuracy: 0.7880 - val_Right_accuracy: 0.7920 - val_loss: 1.8431
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - Left_accuracy: 0.7863 - Right_accuracy: 0.7856 - loss: 1.3712 - val_Left_accuracy: 0.7880 - val_Right_accuracy: 0.7920 - val_loss: 1.3229
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - Left_accuracy: 0.7835 - Right_accuracy: 0.8020 - loss: 1.1909 - val_Left_accuracy: 0.7880 - val_Right_accuracy: 0.7920 - val_loss: 1.1236
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - Left_accuracy: 0.7859 - Right_accuracy: 0.7975 - loss: 1.1545 - val_Left_accuracy: 0.7880 - val_Right_accuracy: 0.7920 - val_loss: 1.0761
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - Left_accuracy: 0.7807 - Right_accuracy: 0.8034 - loss: 1.1024 - val_Left_accuracy: 0.7880 - val_Right_accuracy: 0.7920 - val_loss: 1.0843
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3

In [17]:
scs_model.save('/kaggle/working/scs_model.h5')
nfn_model.save('/kaggle/working/nfn_model.h5')
ss_model.save('/kaggle/working/ss_model.h5')